In [ ]:
import cv2
import numpy as np
from cvzone.SelfiSegmentationModule import SelfiSegmentation

#Open a simple image
img=cv2.imread(r"C:\Users\91797\Downloads\driver4.jpg")
segmentor = SelfiSegmentation()
img = segmentor.removeBG(img, (0,0,0), threshold=0.9)

#converting from gbr to hsv color space
img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#skin color range for hsv color space 
HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 
v = cv2.split(HSV_mask)
v = cv2.equalizeHist(v)
HSV_mask = cv2.merge((h,s,v))
HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

#converting from bgr to YCbCr color space
img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
#skin color range for hsv color space 
YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

#img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#RGB_mask = cv2.inRange(img_rgb, (50, 50, 25), (250, 200, 150))
#RGB_mask = cv2.morphologyEx(RGB_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

#merge skin detection (YCbCr and hsv)
#global_mask=cv2.bitwise_and(RGB_mask,(cv2.bitwise_and(YCrCb_mask,HSV_mask)))
global_mask = cv2.bitwise_and(YCrCb_mask, HSV_mask)
global_mask=cv2.medianBlur(global_mask,3)
global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))

HSV_result = HSV_mask
YCrCb_result = YCrCb_mask
global_result = global_mask

#HSV_result = cv2.bitwise_not(HSV_mask)
#YCrCb_result = cv2.bitwise_not(YCrCb_mask)
#global_result=cv2.bitwise_not(global_mask)


#show results
cv2.imshow("1_HSV.jpg",HSV_result)
cv2.imshow("2_YCbCr.jpg",YCrCb_result)
#cv2.imshow("3_RGB_Mask", RGB_mask)
cv2.imshow("3_global_result.jpg",global_result)
cv2.imshow("Image.jpg",img)
cv2.imwrite("1_HSV.jpg",HSV_result)
cv2.imwrite("2_YCbCr.jpg",YCrCb_result)
cv2.imwrite("3_global_result.jpg",global_result)
cv2.waitKey(0)
cv2.destroyAllWindows()  

In [ ]:
import cv2
import numpy as np
from cvzone.SelfiSegmentationModule import SelfiSegmentation

segmentor = SelfiSegmentation()

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 30)
while True:
    success, img = cap.read()
    img_out = segmentor.removeBG(img, (0,0,0), threshold=0.99)
    img_hsv = cv2.cvtColor(img_out, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(img_hsv)
    v = cv2.equalizeHist(v)
    img_hsv_enhanced = cv2.merge((h,s,v))

    cv2.imshow('cap_bgr', img)
    cv2.imshow('cap_hsv', img_hsv)
    cv2.imshow('cap_hsv_enhanced', img_hsv_enhanced)

    cv2.waitKey(0)

cv2.release()
cv2.destroyAllWindows



In [ ]:
!pip install pip

In [ ]:
import cv2
import numpy as np
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation
import os
import dlib

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
cap.set(cv2.CAP_PROP_FPS, 60)

segmentor = SelfiSegmentation()
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/91797/Downloads/shape_predictor_68_face_landmarks.dat")

fpsReader = cvzone.FPS()

while True:
    success, img = cap.read()
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #print(img)
    # Removing the background
    img = segmentor.removeBG(img, (0,0,0), threshold=0.99)
    imgOut = segmentor.removeBG(img, (0,0,0), threshold=0.99)
    #_, frame = cap.read()
    gray = cv2.cvtColor(imgOut, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    #b, g, r = cv2.split(img)
    #wb = b == 255
    #wg = g == 255
    #wr = r == 255

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        landmarks = predictor(gray, face)
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            cv2.circle(imgOut, (x, y), 4, (0, 0, 255), -1)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    h = cv2.equalizeHist(h)
    #s = cv2.equalizeHist(s)
    v = cv2.equalizeHist(v)
    hsv = cv2.merge((h,s,v))
    lower_range = np.array([150,100,100])
    upper_range = np.array([359,255,255])
    hsv = cv2.GaussianBlur(hsv, (1,1), 0)
    hsv = cv2.inRange(hsv, lower_range, upper_range)
    kernel1 = np.ones((1,1), np.uint8)
    kernel2 = np.ones((9,9), np.uint8)
    kernel3 = np.ones((3,3), np.uint8)
    #kernel2 = np.ones((3,3), np.uint8)
    mask = cv2.morphologyEx(hsv, cv2.MORPH_OPEN, kernel1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel2)
    #ret, binary = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)
    mask_gradient = cv2.morphologyEx(mask, cv2.MORPH_GRADIENT, kernel3)
    mask_gradient_copy = mask_gradient.copy()
    mask_gradient_copy = cv2.Laplacian(mask_gradient_copy, -1, ksize = 3)
    canny_mask_gradient_copy = cv2.Canny(mask_gradient_copy, 140, 150)
    contours, hierarchy = cv2.findContours(canny_mask_gradient_copy, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    img_contour = cv2.drawContours(canny_mask_gradient_copy, contours, 2, (0,255,255), thickness=3, lineType = cv2.LINE_AA)
    #for face in faces:
     #   x1 = face.left()
      #  y1 = face.top()
       # x2 = face.right()
        #y2 = face.bottom()
        #landmarks = predictor(gray, face)
        #for n in range(48, 68):
        #    x = landmarks.part(n).x
        #    y = landmarks.part(n).y
        #    cv2.circle(mask, (x, y), 4, (0, 0, 255), -1)



    #lower_white = np.array([0,0,168], dtype=np.uint8)
    #upper_white = np.array([172,111,255], dtype= np.uint8)    
    #white_pixels_if_true = np.bitwise_and(wb, np.bitwise_and(wg, wr))
    #img_size = r.size
    #white_pixels_count = np.sum(white_pixels_if_true)
    #white_area_ratio = cv2.inRange(mask, lower_white, upper_white)

    imgStacked = cvzone.stackImages([img, imgOut, mask],3,1)
    _, imgStacked = fpsReader.update(imgStacked, color = (0, 0, 255))
    #if (white_area_ratio == True):
     #   font = cv2.FONT_HERSHEY_SIMPLEX
      #  cv2.putText(imgStacked, 'Drowsy Alert', (50,150), font, 1, (0,255,255),2,cv2.LINE_4)
    cv2.imshow("Redness of Mouth", imgStacked)
    cv2.imshow("Gradient of mouth", mask_gradient)
    cv2.imshow("Contours", img_contour)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.release()
cv2.destroyAllWindows


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
 
cap = cv2.VideoCapture(0)
 
while(1):
 
    # Take each frame
    _, frame = cap.read()
 
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # define range of red color in HSV
    lower_red = np.array([160,50,50])
    upper_red = np.array([180,255,255])  
 
    #Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
 
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
 
    # we are just adding 2 more channels on the mask so we can stack it along other images
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    kernel1 = np.ones((3,3), np.uint8)
    kernel2 = np.ones((5,5), np.uint8)
    mask_3 = cv2.morphologyEx(mask_3, cv2.MORPH_CLOSE, kernel2)
    mask_3_copy = mask_3.copy()
    mask1 = cv2.medianBlur(mask_3_copy, 3)
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel1)

    mask2 = cv2.GaussianBlur(mask_3_copy, (3,3), 0)
    
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel1)

    gradients_sobelx1 = cv2.Sobel(mask1, -1, 1, 0)
    gradients_sobely1 = cv2.Sobel(mask1, -1, 0, 1)
    gradients_sobelxy1 = cv2.addWeighted(gradients_sobelx1, 0.5, gradients_sobely1, 0.5, 0)

    gradients_sobelx2 = cv2.Sobel(mask2, -1, 1, 0)
    gradients_sobely2 = cv2.Sobel(mask2, -1, 0, 1)
    gradients_sobelxy2 = cv2.addWeighted(gradients_sobelx2, 0.5, gradients_sobely2, 0.5, 0)

    laplacian1 = cv2.Laplacian(mask1, -1)
    
    laplacian2 = cv2.Laplacian(mask2, -1)

    canny = cv2.Canny(mask_3, 100, 150)
    canny = cv2.equalizeHist(canny)

    # stacking up all three images together
    stacked = np.hstack((mask_3,frame,res))
     
    cv2.imshow('Result',cv2.resize(stacked,None,fx=0.8,fy=0.8))
    cv2.imshow("mask1", mask1)
    cv2.imshow("mask2", mask2)

    cv2.imshow("sobel1", gradients_sobelxy1)
    cv2.imshow("sobel2", gradients_sobelxy2)
    
    cv2.imshow("laplacian1", laplacian1)
    cv2.imshow("laplacian2", laplacian2)

    cv2.imshow("canny", canny)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
 
cv2.destroyAllWindows()
cap.release()

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
 
cap = cv2.VideoCapture(0)
 
while(1):
 
    # Take each frame
    _, frame = cap.read()
    
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # define range of red color in HSV
    lower_red = np.array([160,50,50])
    upper_red = np.array([180,255,255])  
 
    #Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
 
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
 
    # we are just adding 2 more channels on the mask so we can stack it along other images
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    mask1 = cv2.medianBlur(mask_3, 3)
    mask2 = cv2.GaussianBlur(mask_3, (3,3), 0)

    gradients_sobelx1 = cv2.Sobel(mask1, -1, 1, 0)
    gradients_sobely1 = cv2.Sobel(mask1, -1, 0, 1)
    gradients_sobelxy1 = cv2.addWeighted(gradients_sobelx1, 0.5, gradients_sobely1, 0.5, 0)
    gradients_sobelxy1 = cv2.cvtColor(gradients_sobelxy1, cv2.COLOR_BGR2GRAY)
    ret1, binary1 = cv2.threshold(gradients_sobelxy1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours1, hierarchy1 = cv2.findContours(binary1, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    contour_gradients_sobel_medianBlur = cv2.drawContours(gradients_sobelxy1, contours1, -1, (0,255,0), thickness=2, lineType = cv2.LINE_AA)

    gradients_sobelx2 = cv2.Sobel(mask2, -1, 1, 0)
    gradients_sobely2 = cv2.Sobel(mask2, -1, 0, 1)
    gradients_sobelxy2 = cv2.addWeighted(gradients_sobelx2, 0.5, gradients_sobely2, 0.5, 0)
    gradients_sobelxy2 = cv2.cvtColor(gradients_sobelxy2, cv2.COLOR_BGR2GRAY)
    ret2, binary2 = cv2.threshold(gradients_sobelxy2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours2, hierarchy2 = cv2.findContours(binary2, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    contour_gradients_sobel_GaussianBlur = cv2.drawContours(gradients_sobelxy2, contours2, -1, (0,255,0), thickness=2, lineType = cv2.LINE_AA)

    laplacian1 = cv2.Laplacian(mask1, -1)
    laplacian1 = cv2.cvtColor(laplacian1, cv2.COLOR_BGR2GRAY)
    ret3, binary3 = cv2.threshold(laplacian1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours3, hierarchy3 = cv2.findContours(binary3, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    contour_gradients_laplacian_medianBlur = cv2.drawContours(laplacian1, contours3, -1, (0,255,0), thickness=2, lineType = cv2.LINE_AA)
    laplacian2 = cv2.Laplacian(mask2, -1)
    laplacian2 = cv2.cvtColor(laplacian2, cv2.COLOR_BGR2GRAY)  
    ret4, binary4 = cv2.threshold(laplacian2, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours4, hierarchy4 = cv2.findContours(binary4, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    contour_gradients_laplacian_GaussianBlur = cv2.drawContours(laplacian2, contours4, -1, (0,255,0), thickness=2, lineType = cv2.LINE_AA)

    canny = cv2.Canny(mask_3, 100, 150)
    canny = cv2.equalizeHist(canny)
    ret5, binary5 = cv2.threshold(canny, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    contours5, hierarchy5 = cv2.findContours(binary5, mode = cv2.RETR_EXTERNAL, method = cv2.CHAIN_APPROX_NONE)
    contour_canny = cv2.drawContours(canny, contours5, -1, (0,255,0), thickness=2, lineType = cv2.LINE_AA)
    # stacking up all three images together
    stacked = np.hstack((mask_3,frame,res))
     
    cv2.imshow('Result',cv2.resize(stacked,None,fx=0.8,fy=0.8))
    cv2.imshow("mask1", mask1)
    cv2.imshow("mask2", mask2)

    cv2.imshow("sobel1", gradients_sobelxy1)
    cv2.imshow("sobel2", gradients_sobelxy2)
    cv2.imshow("contour_gradients_sobel_medianBlur", contour_gradients_sobel_medianBlur)
    cv2.imshow("contour_gradients_sobel_GaussianBlur", contour_gradients_sobel_GaussianBlur)

    cv2.imshow("laplacian1", laplacian1)
    cv2.imshow("laplacian2", laplacian2)
    cv2.imshow("contours_gradients_laplacian_medianBlur", contour_gradients_laplacian_medianBlur)
    cv2.imshow("contour_gradients_laplacian_GaussianBlur", contour_gradients_laplacian_GaussianBlur)

    cv2.imshow("canny", canny)
    cv2.imshow("contour_canny", contour_canny)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
 
cv2.destroyAllWindows()
cap.release()

In [ ]:
# this cell is to detect mouth contours in low light background

import cv2
import matplotlib.pyplot as plt
import numpy as np
 
cap = cv2.VideoCapture(0)
 
while(1):
 
    # Take each frame
    _, frame = cap.read()

    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    contrast = 1.25
    brightness = 150
    frame[:,:,2] = np.clip(contrast * frame[:,:,2] + brightness, 0, 255)
    frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # define range of red color in HSV
    lower_red = np.array([160,50,60])
    upper_red = np.array([180,255,255])  
 
    #Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
 
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
 
    # we are just adding 2 more channels on the mask so we can stack it along other images
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    kernel1 = np.ones((3,3), np.uint8)
    kernel2 = np.ones((5,5), np.uint8)
    mask_3 = cv2.morphologyEx(mask_3, cv2.MORPH_CLOSE, kernel2)
    mask_3_copy = mask_3.copy()
    mask1 = cv2.medianBlur(mask_3_copy, 3)
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel1)

    mask2 = cv2.GaussianBlur(mask_3_copy, (3,3), 0)
    
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel1)

    gradients_sobelx1 = cv2.Sobel(mask1, -1, 1, 0)
    gradients_sobely1 = cv2.Sobel(mask1, -1, 0, 1)
    gradients_sobelxy1 = cv2.addWeighted(gradients_sobelx1, 0.5, gradients_sobely1, 0.5, 0)

    gradients_sobelx2 = cv2.Sobel(mask2, -1, 1, 0)
    gradients_sobely2 = cv2.Sobel(mask2, -1, 0, 1)
    gradients_sobelxy2 = cv2.addWeighted(gradients_sobelx2, 0.5, gradients_sobely2, 0.5, 0)

    laplacian1 = cv2.Laplacian(mask1, -1)
    
    laplacian2 = cv2.Laplacian(mask2, -1)

    canny = cv2.Canny(mask_3, 100, 150)
    canny = cv2.equalizeHist(canny)
    canny = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel2)

    # stacking up all three images together
    stacked = np.hstack((mask_3,frame,res))
     
    cv2.imshow('Result',cv2.resize(stacked,None,fx=0.8,fy=0.8))
    cv2.imshow("mask1", mask1)
    cv2.imshow("mask2", mask2)

    cv2.imshow("sobel1", gradients_sobelxy1)
    cv2.imshow("sobel2", gradients_sobelxy2)
    
    cv2.imshow("laplacian1", laplacian1)
    cv2.imshow("laplacian2", laplacian2)

    cv2.imshow("canny", canny)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
 
cv2.destroyAllWindows()
cap.release()

In [ ]:
# A video was captured by a low quality camera from the dashboard of the car. The driver was monitored through this camera for 1 hour and my algorithm was tested
# on that video. The algorithm gives really good results with accurate contour detection

import cv2
import matplotlib.pyplot as plt
import numpy as np
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation
import os
import dlib

cap = cv2.VideoCapture("C:/Users/91797/Downloads/output.avi")
segmentor = SelfiSegmentation()
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/91797/Downloads/shape_predictor_68_face_landmarks.dat")
 
while(1):
 
    # Take each frame
    _, frame = cap.read()
    
    height, width, layers = frame.shape
    new_h = int(height/2)
    new_w = int(width/4)
    frame = cv2.resize(frame, (new_h, new_w))
    frame = segmentor.removeBG(frame, (0,0,0), threshold=0.3)
 
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # define range of red color in HSV
    lower_red = np.array([160,50,50])
    upper_red = np.array([180,255,255])  
 
    #Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
 
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
 
    # we are just adding 2 more channels on the mask so we can stack it along other images
    mask_3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

    kernel1 = np.ones((3,3), np.uint8)
    kernel2 = np.ones((5,5), np.uint8)
    kernel3 = np.ones((9,9), np.uint8)
    mask_3 = cv2.morphologyEx(mask_3, cv2.MORPH_CLOSE, kernel2)
    mask_3_copy = mask_3.copy()
    mask1 = cv2.medianBlur(mask_3_copy, 3)
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, kernel2)
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, kernel1)

    mask2 = cv2.GaussianBlur(mask_3_copy, (3,3), 0)
    
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_OPEN, kernel2)
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernel1)


    gradients_sobelx1 = cv2.Sobel(mask1, -1, 1, 0)
    gradients_sobely1 = cv2.Sobel(mask1, -1, 0, 1)
    gradients_sobelxy1 = cv2.addWeighted(gradients_sobelx1, 0.5, gradients_sobely1, 0.5, 0)

    gradients_sobelx2 = cv2.Sobel(mask2, -1, 1, 0)
    gradients_sobely2 = cv2.Sobel(mask2, -1, 0, 1)
    gradients_sobelxy2 = cv2.addWeighted(gradients_sobelx2, 0.5, gradients_sobely2, 0.5, 0)

    laplacian1 = cv2.Laplacian(mask1, -1)
    
    laplacian2 = cv2.Laplacian(mask2, -1)

    canny = cv2.Canny(mask_3, 100, 150)
    canny = cv2.equalizeHist(canny)

    # stacking up all three images together
    stacked = np.hstack((mask_3,frame,res))
     
    cv2.imshow('Result',cv2.resize(stacked,None,fx=0.8,fy=0.8))
    cv2.imshow("mask1", mask1)
    cv2.imshow("mask2", mask2)

    cv2.imshow("sobel1", gradients_sobelxy1)
    cv2.imshow("sobel2", gradients_sobelxy2)
    
    cv2.imshow("laplacian1", laplacian1)
    cv2.imshow("laplacian2", laplacian2)

    cv2.imshow("canny", canny)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
 
cv2.destroyAllWindows()
cap.release()
